In [1]:
import sympy as sym 
import math
import cmath
import cython as cp
import numpy as np
import pytearcat as pt


In [2]:
from sympy import *

In [3]:
from sympy.physics import *
from sympy.tensor.tensor import *
from sympy.physics.hep.gamma_matrices import GammaMatrix as G, LorentzIndex

In [4]:
 n, m, c = symbols('n m c', integer=True)

In [5]:
i, j = symbols('i j', cls=Idx)

In [6]:
var('t, x, y, z,', real=True)

(t, x, y, z)

In [7]:
phi = symbols('phi')

phi
S= Function('S')(t,x,y,z)

In [8]:
A0= Function('A0')
A1= Function('A1')
A2= Function('A2')
A3= Function('A3')
A4= Function('A4')
A5= Function('A5')

In [9]:
A=(A0, A1, A2, A3, A4, A5)
indices = [0, 1, 2, 3, 4]


In [10]:
A

(A0, A1, A2, A3, A4, A5)

In [11]:
phi

phi

In [12]:
phi = 0
for k in indices:
    m= exp(I*S*c**(2))*(A[k](t,x,y,z)*c**(-k))
    phi = phi +m

In [13]:
phi

A0(t, x, y, z)*exp(I*c**2*S(t, x, y, z)) + A1(t, x, y, z)*exp(I*c**2*S(t, x, y, z))/c + A2(t, x, y, z)*exp(I*c**2*S(t, x, y, z))/c**2 + A3(t, x, y, z)*exp(I*c**2*S(t, x, y, z))/c**3 + A4(t, x, y, z)*exp(I*c**2*S(t, x, y, z))/c**4

In [14]:
chi = 0
for k in indices:
    m= exp(-I*S*c**(2))*(A[k](t,x,y,z)*c**(-k))
    chi = chi +m

In [15]:
diff(phi,x)

I*c**2*A0(t, x, y, z)*exp(I*c**2*S(t, x, y, z))*Derivative(S(t, x, y, z), x) + I*c*A1(t, x, y, z)*exp(I*c**2*S(t, x, y, z))*Derivative(S(t, x, y, z), x) + I*A2(t, x, y, z)*exp(I*c**2*S(t, x, y, z))*Derivative(S(t, x, y, z), x) + exp(I*c**2*S(t, x, y, z))*Derivative(A0(t, x, y, z), x) + I*A3(t, x, y, z)*exp(I*c**2*S(t, x, y, z))*Derivative(S(t, x, y, z), x)/c + exp(I*c**2*S(t, x, y, z))*Derivative(A1(t, x, y, z), x)/c + I*A4(t, x, y, z)*exp(I*c**2*S(t, x, y, z))*Derivative(S(t, x, y, z), x)/c**2 + exp(I*c**2*S(t, x, y, z))*Derivative(A2(t, x, y, z), x)/c**2 + exp(I*c**2*S(t, x, y, z))*Derivative(A3(t, x, y, z), x)/c**3 + exp(I*c**2*S(t, x, y, z))*Derivative(A4(t, x, y, z), x)/c**4

In [16]:
Lorentz = TensorIndexType('Lorentz', dummy_name='L', dim=4, eps_dim=4, metric_symmetry=0, metric_name='metric')
mu = TensorIndex('mu', Lorentz, is_up=False)
nu, rho = tensor_indices('nu, rho', Lorentz)


In [17]:
G(nu)

GammaMatrix(nu)